In [8]:
import pandas as pd
import numpy as np
import networkx as nx
import itertools
import random
from operator import itemgetter
import matplotlib.pyplot as plt

### 1. Create a directed graph

In [9]:
graph = nx.DiGraph()

In [10]:
with open("connections.txt", "r") as f:
    edges = [e.strip().split(" ") for e in f.readlines()]

edges[:5]

[['0', '1'], ['0', '2'], ['0', '3'], ['0', '4'], ['0', '5']]

In [11]:
graph.add_edges_from(edges)

### 2. Which nodes are bridges?

In [12]:
def find_bridge_nodes(graph):
    bridge_nodes = set()

    visited = set()
    parent = dict()
    low = dict()
    disc = dict()

    def dfs(u):
        nonlocal time
        children = 0
        visited.add(u)
        disc[u] = low[u] = time
        time += 1

        for v in graph.successors(u):
            if v not in visited:
                children += 1
                parent[v] = u
                dfs(v)

                low[u] = min(low[u], low[v])

                if low[v] > disc[u]:
                    bridge_nodes.add(u)
                    bridge_nodes.add(v)
            elif v != parent[u]:
                low[u] = min(low[u], disc[v])

    time = 0
    for node in graph.nodes():
        if node not in visited:
            parent[node] = None
            dfs(node)

    return bridge_nodes


len((bridge_edges := find_bridge_nodes(graph)))

1606

In [13]:
len(graph.nodes)

4039

### 3. Show the density of the graph

In [14]:
nx.density(graph)

0.0054099817517196435

the density value is quite low

### 4. Show which nodes have the highest and lowest number of connections.

In [15]:
print("Highest: ", sorted(graph.nodes, key=graph.degree)[-5:])

Highest:  ['0', '3437', '1912', '1684', '107']


In [16]:
print("Lowest: ", sorted(graph.nodes, key=graph.degree)[:5])

Lowest:  ['11', '12', '15', '18', '37']


### 5. Show which nodes have the highest incoming and outgoing connections

In [17]:
in_degrees = dict(graph.in_degree())
out_degrees = dict(graph.out_degree())

sorted_in = sorted(list(in_degrees.keys()), key=lambda x: in_degrees[x])
sorted_out = sorted(list(out_degrees.keys()), key=lambda x: out_degrees[x])

In [18]:
print("Highest in: ", sorted_in[-5:])
print("Highest out: ", sorted_out[-5:])

Highest in:  ['1827', '2611', '1800', '2543', '1888']
Highest out:  ['0', '3437', '1912', '1684', '107']


In [19]:
print("Lowest in: ", sorted_in[:5])
print("Lowest out: ", sorted_out[:5])

Lowest in:  ['0', '686', '1', '2', '3']
Lowest out:  ['11', '12', '15', '18', '37']


### 6. Show which nodes have the highest closeness, betweenness, and eigenvector
Interpret your findings

In [20]:
betweenness = nx.centrality.betweenness_centrality(graph)
highest_betweenness_nodes = sorted(graph.nodes, key=betweenness.get)

highest_betweenness_nodes[-5:]

['1405', '563', '1718', '1912', '1684']

In [21]:
closeness = nx.centrality.closeness_centrality(graph)
highest_closeness_nodes = sorted(graph.nodes, key=closeness.get)

highest_closeness_nodes[-5:]

['2543', '2643', '2629', '2649', '2642']

In [22]:
eigenvector = nx.centrality.eigenvector_centrality(graph, max_iter=90000)
highest_eigenvector_nodes = sorted(graph.nodes, key=eigenvector.get)

highest_eigenvector_nodes[-5:]

['2631', '2638', '2646', '2654', '2655']

### 7. Implement a community detection algorithm on the directed graph and show how many communities were created.

In [36]:
community_size=5

for i in range(community_size):
    print(i)
    # Convert the graph to an undirected graph
    undirected_graph = graph.to_undirected()
    edge_betweenness = nx.edge_betweenness_centrality(graph).items()
    edge_to_delete = sorted(edge_betweenness, key=lambda pair: -pair[1])[0][0]
    
    graph.remove_edge(*edge_to_delete)


    # Calculate connected components
    communities = list(nx.connected_components(undirected_graph))
    
    # Calculate modularity
    modularity = nx.community.quality.modularity(undirected_graph, communities)
    print("Modularity:", modularity)

0
Modularity: 1.1102230246251565e-16
1
Modularity: 0.0
2
Modularity: 0.0
3
Modularity: 0.0
4
Modularity: 0.0


No modularity: the network is too connected for me to find anything with my computational resources